# Exercise 1

Building and simulation pathways using COBRApy
 
![alt text](histidine_pathway_encyclopedia_microbiology.png)


Simulating histidine biosynthesis
 1. Initiate empty model 
 2. Add the metabolites that participat in the pathway
 3. Populated the model the *E. coli* pathway to synthesize L-histidine from Ribose

In [1]:
import cobra
import escher

In [2]:
builder = escher.Builder(map_json='histidine_ribose_map.json')
builder.display_in_notebook()

In [3]:
# Initiate empty model
model = cobra.Model()

# with no reactions
print('Number of reactions: ' + str(len(model.reactions)))
print('Number of metabolites: ' + str(len(model.metabolites)))

Number of reactions: 0
Number of metabolites: 0


In [4]:
from histidine_pathway import metabolites_and_info, reactions_and_info

In [5]:
import pandas as pd
pd.DataFrame.from_dict(reactions_and_info).T.to_csv('reactions_and_info.csv')

---------
## Adding metabolites in pathway

In [6]:
print(metabolites_and_info['his__L_c'])

{'charge': 0, 'compartment': 'c', 'formula': 'C6H9N3O2', 'name': 'L-Histidine'}


In [7]:
# Start with adding a single metabolite with the above associated information
met = cobra.Metabolite(id='his__L_c', compartment='c', charge=0, formula='C6H9N3O2', name='L-histidine')

In [8]:
# Still no metabolites in model
print('Number of metabolites:' + str(len(model.metabolites)))

Number of metabolites:0


In [9]:
# Need to manually add it
model.add_metabolites(met)
print('Number of metabolites:' + str(len(model.metabolites)))

Number of metabolites:1


In [10]:
# Can access or edit the metabolite by either
model.metabolites.his__L_c
# or
model.metabolites.get_by_id('his__L_c')

Metabolite identifier,his__L_c
Name,L-histidine
Memory address,0x07f7fead5e160
Formula,C6H9N3O2
Compartment,c
In 0 reaction(s),


In [11]:
# Add all the remaining metabolites involved in the pathway
for met_id, info in metabolites_and_info.items():
    met = cobra.Metabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    model.add_metabolites(met)

In [12]:
# Need to manually add it
print('Number of metabolites:' + str(len(model.metabolites)))

Number of metabolites:29


-------
## Adding reactions in pathway

In [13]:
print(reactions_and_info['HISTD'])

{'lower_bound': 0.0, 'name': 'histidinol dehydrogenase', 'stoichiometry': {'h2o_c': -1.0, 'h_c': 3.0, 'his__L_c': 1.0, 'histd_c': -1.0, 'nad_c': -2.0, 'nadh_c': 2.0}, 'upper_bound': 1000.0}


In [14]:
rxn = cobra.Reaction(id='HISTD', name='histidinol dehydrogenase')
rxn.upper_bound = 1000. # this is effectively infinity
rxn.lower_bound = 0 # irreversible
rxn.add_metabolites({'h2o_c': -1.0, 'h_c': 3.0, 'his__L_c': 1.0, 
                    'histd_c': -1.0, 'nad_c': -2.0, 'nadh_c': 2.0})

ValueError: Reaction 'HISTD' does not belong to a model. Either add the reaction to a model or use Metabolite objects instead of strings as keys.

In [15]:
# Need to add reaction to model to access metabolites we have added
model.add_reactions([rxn])
rxn.add_metabolites({'h2o_c': -1.0, 'h_c': 3.0, 'his__L_c': 1.0, 
                    'histd_c': -1.0, 'nad_c': -2.0, 'nadh_c': 2.0})
print(rxn.reaction)

h2o_c + histd_c + 2.0 nad_c --> 3.0 h_c + his__L_c + 2.0 nadh_c


In [16]:
# Upper and lower bound sets a reversibility on reactions
rxn.lower_bound = -1000
print(rxn.reaction)
# Return it back to irreversible
rxn.lower_bound = 0

h2o_c + histd_c + 2.0 nad_c <=> 3.0 h_c + his__L_c + 2.0 nadh_c


In [17]:
# Optimize
model.optimize()

,fluxes,reduced_costs
HISTD,0.0,0.0


Add boundary reactions to act as source and sink for ribose and histidine

In [18]:
rxn = cobra.Reaction('EX_rib__D_e')
model.add_reaction(rxn)
rxn.add_metabolites({'rib__D_e': -1})
rxn.lower_bound = -10 # by convention negative exchange flux = uptake

rxn = cobra.Reaction('EX_his__L_e')
model.add_reaction(rxn)
rxn.add_metabolites({'his__L_e': -1})

model.optimize()

,fluxes,reduced_costs
HISTD,0.0,0.0
EX_rib__D_e,0.0,0.0
EX_his__L_e,0.0,0.0


**Still no flux through the reaction?  Why?**

Flux balance analysis assumes that the system is opperating at steady state (meaning the concentration of the metabolites in the system do not increase or decrease over time). As a consequence of this, each metabolite that participates in the pathway must be created and consumed at the same rate. 

In [36]:
iml = cobra.io.load_json_model('iML1515.json')
import cobra.test
#iml = cobra.test.create_test_model('ecoli')
iml.reactions.EX_glc__D_e.lower_bound = 0
iml.reactions.EX_o2_e.lower_bound = -1000
iml.reactions.EX_rib__D_e.lower_bound = -10
iml.reactions.ATPM.lower_bound = 0
iml.objective = 'EX_his__L_e'
#iml.optimize()
solution = cobra.flux_analysis.pfba(iml)

In [37]:
# Find number of reactions that must be active to produce histidine from ribose
active = 0
enzymatic = 0
for r in iml.reactions:
    if abs(r.x) > 1e-10:
        active += 1
        if r.gene_reaction_rule not in  ['s0001', '']:
            enzymatic += 1
            
print('Maximum histidine production flux:', round(solution.fluxes['EX_his__L_e'], 3))
print('Reactions active:', active)
print('Enzymatic reactions active:', enzymatic)

Maximum histidine production flux: 6.638
Reactions active: 78
Enzymatic reactions active: 69


In [38]:
builder = escher.Builder(map_json='primer_iML_map.json', 
                         reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'value', 'value': .01, 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#0000dd', 'size': 20}])
builder.reaction_data = solution.fluxes.to_dict()
builder.display_in_notebook()

In [46]:
iml.reactions.EX_o2_e.lower_bound = 0
solution = cobra.flux_analysis.pfba(iml)

# Find number of reactions that must be active to produce histidine from ribose
active = 0
enzymatic = 0
for r in iml.reactions:
    if abs(r.x) > 1e-10:
        active += 1
        if r.gene_reaction_rule not in  ['s0001', '']:
            enzymatic += 1
print('Maximum histidine production flux:', round(solution.fluxes['EX_his__L_e'], 3))
print('Anaerobic reactions active:', active)
print('Anaerobic Enzymatic reactions active:', enzymatic)

Maximum histidine production flux: 0.915
Anaerobic reactions active: 68
Anaerobic Enzymatic reactions active: 58


In [48]:
builder = escher.Builder(map_json='primer_iML_map.json', 
                         reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'value', 'value': .01, 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#0000dd', 'size': 20}])
builder.reaction_data = solution.fluxes.to_dict()
builder.display_in_notebook()